In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from pathlib import Path
import json

# Step 1: Define paths and hyperparameters
DATA_ROOT = "/content/drive/MyDrive/NTU-Roselab-Dataset"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 8
LEARNING_RATE = 1e-4
N_FOLDS = 5
MODEL_NAME = "Laplacian_MobileNetV2"
OUTPUT_DIR = f"/content/drive/MyDrive/Recapture_Photo_Detection/{MODEL_NAME}/results"
SPLIT_DIR = "/content/drive/MyDrive/Recapture_Photo_Detection"
PREPROCESSING = "Laplacian"
HYPERPARAMETERS = {
    "learning_rate": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "optimizer": "Adam",
    "epochs": EPOCHS,
    "n_folds": N_FOLDS,
    "dropout_rate": 0.5
}

# Step 2: Mount Google Drive and verify dataset path
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ImportError:
    raise ImportError("This script must be run in Google Colab with Google Drive mounted.")
if not os.path.exists(DATA_ROOT):
    raise FileNotFoundError(f"Dataset directory {DATA_ROOT} does not exist. Please check the path.")

# Step 3: Check dataset balance
def check_dataset_balance(data_root):
    originals_path = os.path.join(data_root, 'originals')
    recaptures_path = os.path.join(data_root, 'recaptures')
    originals_count = sum(len(files) for _, _, files in os.walk(originals_path))
    recaptures_count = sum(len(files) for _, _, files in os.walk(recaptures_path))
    print(f"Dataset Balance: {originals_count} originals, {recaptures_count} recaptures")
    return originals_count, recaptures_count

originals_count, recaptures_count = check_dataset_balance(DATA_ROOT)

# Step 4: Define Laplacian preprocessing function
@tf.function
def laplacian_preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    gray = tf.image.rgb_to_grayscale(img)
    lap = tf.image.sobel_edges(gray)
    lap = tf.reduce_sum(tf.square(lap), axis=-1)
    lap = tf.sqrt(lap + 1e-6)
    lap = tf.image.per_image_standardization(lap)
    gray = tf.image.per_image_standardization(gray)
    combined = tf.concat([gray, lap, lap], axis=-1)
    return combined, label

# Step 5: Load and split dataset (single train-test split)
dataset = image_dataset_from_directory(
    DATA_ROOT,
    labels='inferred',
    label_mode='binary',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

# Convert dataset to NumPy arrays for cross-validation
images, labels = [], []
for img_batch, label_batch in dataset:
    images.append(img_batch.numpy())
    labels.append(label_batch.numpy())
images = np.concatenate(images, axis=0)
labels = np.concatenate(labels, axis=0).flatten()

# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create directory for train-test split
Path(SPLIT_DIR).mkdir(parents=True, exist_ok=True)

# Save train-test split for consistency across models
np.save(os.path.join(SPLIT_DIR, 'X_train.npy'), X_train)
np.save(os.path.join(SPLIT_DIR, 'X_test.npy'), X_test)
np.save(os.path.join(SPLIT_DIR, 'y_train.npy'), y_train)
np.save(os.path.join(SPLIT_DIR, 'y_test.npy'), y_test)

# Create test dataset
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE).map(laplacian_preprocess).prefetch(tf.data.AUTOTUNE)

# Step 6: Define function to create MobileNetV2 model
def create_model():
    base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
    for layer in base.layers[:-20]:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(HYPERPARAMETERS['dropout_rate'])(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(base.input, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 7: Convert NumPy types to JSON-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Step 8: Define function to save results
def save_model_results(model, dataset, history, model_name, output_dir, fold=None, preprocessing='None', hyperparameters=None):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    fold_str = f"_fold_{fold}" if fold is not None else ""

    # Evaluate on dataset
    y_true, y_pred = [], []
    for imgs, labels in dataset:
        preds = (model.predict(imgs, verbose=0) > 0.5).astype(int)
        y_true.extend(labels.numpy().astype(int))
        y_pred.extend(preds.flatten())

    # Check prediction distribution
    originals_pred = sum(1 for p in y_pred if p == 0)
    recaptures_pred = sum(1 for p in y_pred if p == 1)
    print(f"Predictions {fold_str}: {originals_pred} originals, {recaptures_pred} recaptures")

    # Classification report
    class_report = classification_report(y_true, y_pred, target_names=['originals', 'recaptured'], output_dict=True)
    class_report_df = pd.DataFrame(class_report).transpose()
    class_report_df.to_csv(f'{output_dir}/{model_name}_classification_report{fold_str}.csv')

    # Confusion matrix (detailed)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['originals', 'recaptured'], yticklabels=['originals', 'recaptured'])
    plt.title(f'Confusion Matrix - {model_name}{fold_str}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.png')
    plt.close()

    # Save confusion matrix as CSV
    cm_df = pd.DataFrame(cm, index=['True_originals', 'True_recaptured'], columns=['Pred_originals', 'Pred_recaptured'])
    cm_df.to_csv(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.csv')

    # Model summary (only for final model)
    if fold is None:
        summary_file = f'{output_dir}/{model_name}_summary.txt'
        with open(summary_file, 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))

    # Calculate total and trainable parameters
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    # Aggregate results
    results = {
        'Model': model_name,
        'Preprocessing': preprocessing,
        'Accuracy': class_report['accuracy'],
        'Total_Parameters': total_params,
        'Trainable_Parameters': trainable_params,
        'Fold': fold if fold is not None else 'Final'
    }
    if hyperparameters:
        results.update(hyperparameters)
    for label, metrics in class_report.items():
        if isinstance(metrics, dict):
            results.update({
                f'Precision_{label}': metrics['precision'],
                f'Recall_{label}': metrics['recall'],
                f'F1-Score_{label}': metrics['f1-score'],
                f'Support_{label}': metrics['support']
            })

    # Convert NumPy types to JSON-serializable types
    results = {k: convert_to_serializable(v) for k, v in results.items()}

    # Save results to JSON
    with open(f'{output_dir}/{model_name}_results{fold_str}.json', 'w') as f:
        json.dump(results, f, indent=4)

    # Plot and save accuracy/loss curves
    if history is not None:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Accuracy Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Loss Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/{model_name}_accuracy_loss_curve{fold_str}.png')
        plt.close()

    # Save model weights (only for final model)
    if fold is None:
        model.save(f'{output_dir}/{model_name}_model.h5')

    return results

# Step 9: Perform 5-fold cross-validation
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
fold_results = []
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"\nTraining Fold {fold + 1}/{N_FOLDS}")

    # Create train and validation datasets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

    train_ds = tf.data.Dataset.from_tensor_slices((X_fold_train, y_fold_train)).batch(BATCH_SIZE).map(laplacian_preprocess).prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_fold_val, y_fold_val)).batch(BATCH_SIZE).map(laplacian_preprocess).prefetch(tf.data.AUTOTUNE)

    # Create and train model with early stopping
    model = create_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping], verbose=1)

    # Save results for this fold
    results = save_model_results(
        model, val_ds, history, MODEL_NAME, OUTPUT_DIR,
        fold=fold + 1, preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
    )
    fold_results.append(results)

# Step 10: Train final model on full training set
print("\nTraining final model on full training set")
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).map(laplacian_preprocess).prefetch(tf.data.AUTOTUNE)
model = create_model()
history = model.fit(train_ds, epochs=EPOCHS, verbose=1)

# Step 11: Evaluate final model on test set and save results
results = save_model_results(
    model, test_ds, history, MODEL_NAME, OUTPUT_DIR,
    preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
)
print(f"Final Results for {MODEL_NAME}:", results)

# Step 12: Aggregate cross-validation results
if fold_results:
    fold_df = pd.DataFrame(fold_results)
    mean_results = {
        'Model': MODEL_NAME,
        'Preprocessing': PREPROCESSING,
        'Mean_Accuracy': fold_df['Accuracy'].mean(),
        'Std_Accuracy': fold_df['Accuracy'].std(),
        'Mean_Precision_recaptured': fold_df['Precision_recaptured'].mean(),
        'Mean_Recall_recaptured': fold_df['Recall_recaptured'].mean(),
        'Mean_F1-Score_recaptured': fold_df['F1-Score_recaptured'].mean(),
        'Mean_Total_Parameters': fold_df['Total_Parameters'].mean(),
        'Mean_Trainable_Parameters': fold_df['Trainable_Parameters'].mean()
    }
    # Convert NumPy types in mean_results
    mean_results = {k: convert_to_serializable(v) for k, v in mean_results.items()}
    with open(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_summary.json', 'w') as f:
        json.dump(mean_results, f, indent=4)
    fold_df.to_csv(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_results.csv', index=False)
    print("\nCross-Validation Summary:", mean_results)

Mounted at /content/drive
Dataset Balance: 1202 originals, 1199 recaptures
Found 2401 files belonging to 2 classes.

Training Fold 1/5


/tmp/ipython-input-1142743286.py:109: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.5907 - loss: 0.7342 - val_accuracy: 0.6771 - val_loss: 0.6037
Epoch 2/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.8605 - loss: 0.3345 - val_accuracy: 0.7240 - val_loss: 0.5893
Epoch 3/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.9383 - loss: 0.1954 - val_accuracy: 0.7526 - val_loss: 0.5738
Epoch 4/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9763 - loss: 0.1059 - val_accuracy: 0.7552 - val_loss: 0.5821
Epoch 5/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.9927 - loss: 0.0525 - val_accuracy: 0.7708 - val_loss: 0.5831
Epoch 6/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.9992 - loss: 0.0277 - val_accuracy: 0.7734 - val_loss: 0.6158
Predictions _fold_1: 179 originals, 205 recaptures

Training Fold 2/5


/tmp/ipython-input-1142743286.py:109: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


Epoch 1/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 113s 1s/step - accuracy: 0.5930 - loss: 0.7266 - val_accuracy: 0.7240 - val_loss: 0.5540
Epoch 2/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.8688 - loss: 0.3380 - val_accuracy: 0.7656 - val_loss: 0.5117
Epoch 3/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9533 - loss: 0.1900 - val_accuracy: 0.7865 - val_loss: 0.5245
Epoch 4/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.9840 - loss: 0.1048 - val_accuracy: 0.7839 - val_loss: 0.5149
Epoch 5/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.9959 - loss: 0.0494 - val_accuracy: 0.7969 - val_loss: 0.5191
Predictions _fold_2: 178 originals, 206 recaptures

Training Fold 3/5


...

/tmp/ipython-input-1142743286.py:109: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


Epoch 1/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.5804 - loss: 0.7507 - val_accuracy: 0.6484 - val_loss: 0.7035
Epoch 2/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.8872 - loss: 0.3225 - val_accuracy: 0.6979 - val_loss: 0.6676
Epoch 3/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 136s 1s/step - accuracy: 0.9510 - loss: 0.1959 - val_accuracy: 0.7526 - val_loss: 0.5277
Epoch 4/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.9796 - loss: 0.1101 - val_accuracy: 0.7865 - val_loss: 0.4836
Epoch 5/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9953 - loss: 0.0502 - val_accuracy: 0.7917 - val_loss: 0.5191
Epoch 6/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 100s 1s/step - accuracy: 0.9982 - loss: 0.0286 - val_accuracy: 0.8099 - val_loss: 0.4789
Epoch 7/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 1.0000 - loss: 0.0162 - val_accuracy: 0.8307 - val_loss: 0.4914
Epoch 8/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 1.0000 - loss: 0.0095 - val_accuracy: 0.8307 - val_loss: 

/tmp/ipython-input-1142743286.py:109: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


Epoch 1/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 118s 886ms/step - accuracy: 0.5856 - loss: 0.7480
Epoch 2/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 103s 856ms/step - accuracy: 0.8743 - loss: 0.3242
Epoch 3/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 108s 894ms/step - accuracy: 0.9527 - loss: 0.1729
Epoch 4/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 107s 883ms/step - accuracy: 0.9883 - loss: 0.0860
Epoch 5/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 142s 887ms/step - accuracy: 0.9940 - loss: 0.0430
Epoch 6/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 112s 927ms/step - accuracy: 1.0000 - loss: 0.0202
Epoch 7/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 144s 944ms/step - accuracy: 1.0000 - loss: 0.0116
Epoch 8/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 111s 920ms/step - accuracy: 1.0000 - loss: 0.0074
Predictions : 249 originals, 232 recaptures


Final Results for Laplacian_MobileNetV2: {'Model': 'Laplacian_MobileNetV2', 'Preprocessing': 'Laplacian', 'Accuracy': 0.8461538461538461, 'Total_Parameters': 2259265, 'Trainable_Parameters': 1207361, 'Fold': 'Final', 'learning_rate': 0.0001, 'batch_size': 16, 'optimizer': 'Adam', 'epochs': 8, 'n_folds': 5, 'dropout_rate': 0.5, 'Precision_originals': 0.8353413654618473, 'Recall_originals': 0.8630705394190872, 'F1-Score_originals': 0.8489795918367347, 'Support_originals': 241.0, 'Precision_recaptured': 0.8577586206896551, 'Recall_recaptured': 0.8291666666666667, 'F1-Score_recaptured': 0.8432203389830508, 'Support_recaptured': 240.0, 'Precision_macro avg': 0.8465499930757512, 'Recall_macro avg': 0.8461186030428769, 'F1-Score_macro avg': 0.8460999654098927, 'Support_macro avg': 481.0, 'Precision_weighted avg': 0.8465266903156392, 'Recall_weighted avg': 0.8461538461538461, 'F1-Score_weighted avg': 0.8461059521592208, 'Support_weighted avg': 481.0}

Cross-Validation Summary: {'Model': 'Lapla